# TiTiler-CMR: HLS Data Demo

#### Requirements
- folium
- httpx

`!pip install folium httpx earthaccess`

In [1]:
import json
import httpx
import earthaccess

from folium import Map, TileLayer

In [8]:
titiler_endpoint = "https://dev-titiler-cmr.delta-backend.com/api.html"

In [3]:
datasets = earthaccess.search_datasets(doi="10.5067/HLS/HLSL30.002")
ds = datasets[0]
print(ds)

Datasets found: 1
{
  "meta": {
    "revision-id": 51,
    "deleted": false,
    "format": "application/vnd.nasa.cmr.umm+json",
    "provider-id": "LPCLOUD",
    "user-id": "thouska",
    "has-formats": false,
    "associations": {
      "tools": [
        "TL1860232272-LPDAAC_ECS"
      ]
    },
    "s3-links": [
      "s3://lp-prod-protected/HLSL30.020",
      "s3://lp-prod-public/HLSL30.020"
    ],
    "has-spatial-subsetting": false,
    "native-id": "HLSL30V2",
    "has-transforms": false,
    "association-details": {
      "tools": [
        {
          "concept-id": "TL1860232272-LPDAAC_ECS"
        }
      ]
    },
    "has-variables": false,
    "concept-id": "C2021957657-LPCLOUD",
    "revision-date": "2024-03-22T13:26:44.083Z",
    "granule-count": 11486969,
    "has-temporal-subsetting": false,
    "concept-type": "collection"
  },
  "umm": {
    "TilingIdentificationSystems": [
      {
        "TilingIdentificationSystemName": "Military Grid Reference System",
        "Coo

In [4]:
concept_id = ds["meta"]["concept-id"]
print("Concept-Id: ", concept_id)

Concept-Id:  C2021957657-LPCLOUD


In [15]:
import earthaccess
import morecantile

tms = morecantile.tms.get("WebMercatorQuad")

bounds = tms.bounds(62, 44, 7)
xmin, ymin, xmax, ymax = (round(n, 8) for n in bounds)

results = earthaccess.search_data(
    bounding_box=(xmin, ymin, xmax, ymax),
    count=1,
    concept_id=concept_id,
    temporal=("2024-02-11", "2024-02-13"),
)
print("Granules:")
print(results)
print()
print("Example of COGs URL: ")
for link in results[0].data_links(access="direct"):
    print(link)


Granules found: 9
Granules:
[Collection: {'EntryTitle': 'HLS Landsat Operational Land Imager Surface Reflectance and TOA Brightness Daily Global 30m v2.0'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -2.64743819, 'Latitude': 48.6644919}, {'Longitude': -2.21521695, 'Latitude': 49.65006328}, {'Longitude': -3.00027708, 'Latitude': 49.65272281}, {'Longitude': -3.00027162, 'Latitude': 48.66503141}, {'Longitude': -2.64743819, 'Latitude': 48.6644919}]}}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2024-02-12T11:05:26.302Z', 'EndingDateTime': '2024-02-12T11:05:50.181Z'}}
Size(MB): 56.62721920013428
Data: ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T30UWV.2024043T110526.v2.0/HLS.L30.T30UWV.2024043T110526.v2.0.B02.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T30UWV.2024043T110526.v2.0/HLS.L30.T30UWV.2024043T110526.v2.0.B06.tif',

In [6]:
print(results[0])

Collection: {'EntryTitle': 'HLS Landsat Operational Land Imager Surface Reflectance and TOA Brightness Daily Global 30m v2.0'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -2.64743819, 'Latitude': 48.6644919}, {'Longitude': -2.21521695, 'Latitude': 49.65006328}, {'Longitude': -3.00027708, 'Latitude': 49.65272281}, {'Longitude': -3.00027162, 'Latitude': 48.66503141}, {'Longitude': -2.64743819, 'Latitude': 48.6644919}]}}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2024-02-12T11:05:26.302Z', 'EndingDateTime': '2024-02-12T11:05:50.181Z'}}
Size(MB): 56.62721920013428
Data: ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T30UWV.2024043T110526.v2.0/HLS.L30.T30UWV.2024043T110526.v2.0.B02.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T30UWV.2024043T110526.v2.0/HLS.L30.T30UWV.2024043T110526.v2.0.B06.tif', 'https://data.lpdaac.earthda

In [18]:
from titiler.cmr.backend import CMRBackend

with CMRBackend() as backend:
    assets = backend.assets_for_tile(
        x=62,
        y=44,
        z=7,
        bands_regex="B[0-9][0-9]",
        concept_id=concept_id,
        temporal=("2024-02-11", "2024-02-13")
    )

print(assets[0])

Granules found: 9
{'url': {'B02': 's3://lp-prod-protected/HLSL30.020/HLS.L30.T30UWV.2024043T110526.v2.0/HLS.L30.T30UWV.2024043T110526.v2.0.B02.tif', 'B06': 's3://lp-prod-protected/HLSL30.020/HLS.L30.T30UWV.2024043T110526.v2.0/HLS.L30.T30UWV.2024043T110526.v2.0.B06.tif', 'B01': 's3://lp-prod-protected/HLSL30.020/HLS.L30.T30UWV.2024043T110526.v2.0/HLS.L30.T30UWV.2024043T110526.v2.0.B01.tif', 'B07': 's3://lp-prod-protected/HLSL30.020/HLS.L30.T30UWV.2024043T110526.v2.0/HLS.L30.T30UWV.2024043T110526.v2.0.B07.tif', 'B03': 's3://lp-prod-protected/HLSL30.020/HLS.L30.T30UWV.2024043T110526.v2.0/HLS.L30.T30UWV.2024043T110526.v2.0.B03.tif', 'B04': 's3://lp-prod-protected/HLSL30.020/HLS.L30.T30UWV.2024043T110526.v2.0/HLS.L30.T30UWV.2024043T110526.v2.0.B04.tif', 'B05': 's3://lp-prod-protected/HLSL30.020/HLS.L30.T30UWV.2024043T110526.v2.0/HLS.L30.T30UWV.2024043T110526.v2.0.B05.tif', 'B11': 's3://lp-prod-protected/HLSL30.020/HLS.L30.T30UWV.2024043T110526.v2.0/HLS.L30.T30UWV.2024043T110526.v2.0.B11.tif

In [9]:
from IPython.display import IFrame
IFrame("https://dev-titiler-cmr.delta-backend.com/api.html", 900,500)

In [28]:
r = httpx.get(
    "https://dev-titiler-cmr.delta-backend.com/WebMercatorQuad/tilejson.json",
    params = (
        ("concept_id", concept_id),
        # Datetime in form of `start_date/end_date`
        ("datetime", "2024-02-11T00:00:00Z/2024-02-13T23:59:59Z"),
        # We know that the HLS collection dataset is stored as File per Band
        # so we need to pass a `band_regex` option to assign `bands` to each URL
        ("bands_regex", "B[0-9][0-9]"),
        # titiler-cmr can work with both Zarr and COG dataset
        # but we need to tell the endpoints in advance which backend
        # to use
        ("backend", "rasterio"),
        # True Color Image B04,B03,B02
        ("bands", "B04"),
        ("bands", "B03"),
        ("bands", "B02"),
        # The data is in type of Uint16 so we need to apply some
        # rescaling/color_formula in order to create PNGs
        ("color_formula", "Gamma RGB 3.5 Saturation 1.7 Sigmoidal RGB 15 0.35"),
        # We need to set min/max zoom because we don't want to use lowerzoom level (e.g 0)
        # which will results in useless large scale query
        ("minzoom", 8),
        ("maxzoom", 13),
    )
).json()

print(r)

{'tilejson': '2.2.0', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://dev-titiler-cmr.delta-backend.com/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?concept_id=C2021957657-LPCLOUD&datetime=2024-02-11T00%3A00%3A00Z%2F2024-02-13T23%3A59%3A59Z&bands_regex=B%5B0-9%5D%5B0-9%5D&backend=rasterio&color_formula=Gamma+RGB+3.5+Saturation+1.7+Sigmoidal+RGB+15+0.35&bands=B04&bands=B03&bands=B02'], 'minzoom': 8, 'maxzoom': 13, 'bounds': [-180.0, -90.0, 180.0, 90.0], 'center': [0.0, 0.0, 8]}


In [27]:
bounds = r["bounds"]
m = Map(
    location=(47.735842682664014, -3.5006450377327667),
    zoom_start=r["minzoom"]
)

TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="NASA",
).add_to(m)
m